In [ ]:
# For visualisation purpose only (off topic)
from graph import draw_dot

# Lesson 17

 - Start with a stripped-down version of `class Value`
   - it has only `__init__`, `__add__`, and `__mul__`
 - compute `s1` and  `s2`
 - create a `root`, collecting `s1` and `s2` for viz
 - draw `root`
 - init gradients for `s1` and `s2`
 - backprop up to `x1` following two paths
 - get *wrong* result
 - replace `_backward()` assignment with accumulation
 - init gradients for `s1` and `s2`
 - backprop up to `x1` following two paths
 - get *right* result

In [ ]:
class Value:
    def __init__(self, data, _children=(), _op='', label='', color=None):
        self.data = data
        self.grad = 0
        self._prev = set(_children)
        self._op = _op
        self.label = label
        self.color = color
        self._backward = lambda: None
    
    def __repr__(self):
        return f'{self.label or 'Value'}(data={self.data:.2f}, grad={self.grad:.3f})'
    
    def __add__(self, other):
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad = out.grad   # to be fixed
            other.grad = out.grad  # to be fixed
            self.color = 'green'
            other.color = 'green'
        out._backward = _backward

        return out
    
    def __mul__(self, other):
        out = Value(self.data * other.data, (self, other), '×')

        def _backward():
            self.grad = other.data * out.grad  # to be fixed
            other.grad = self.data * out.grad  # to be fixed
            self.color = 'green'
            other.color = 'green'
        out._backward = _backward

        return out


In [ ]:
# Leaves
x1  = Value(+1,   label='x1')
x2  = Value(+0,   label='x2')
w11 = Value(-2,   label='w11')
w12 = Value(+4,   label='w12')
w21 = Value(+3,   label='w21')
w22 = Value(-1,   label='w22')

# s = W @ x
w11x1 = w11 * x1;    w11x1.label = 'w11x1'
w12x2 = w12 * x2;    w12x2.label = 'w12x2'
s1 = w11x1 + w12x2;  s1.label = 's1'
w21x1 = w21 * x1;    w21x1.label = 'w21x1'
w22x2 = w22 * x2;    w22x2.label = 'w22x2'
s2 = w21x1 + w22x2;  s2.label = 's2'

# create an artificial root for graph viz
root = s1 + s2;  root.label = 'root';  root.color = 'black'

In [ ]:
draw_dot(root)

The next cells need to be run backward, from `s{1,2}.grad = {.25,-.125}` upwards

In [ ]:
w21x1._backward()

In [ ]:
s2._backward()

In [ ]:
w11x1._backward()

In [ ]:
s1._backward()

In [ ]:
# Initialise ∂L/∂s1 ∂L/∂s2 with numerical values
s1.grad = 0.25;    s1.color = 'green'
s2.grad = -0.125;  s2.color = 'green'